Install the library and packages

In [ ]:
!pip install matplotlib

In [ ]:
!pip install tensorflow

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from model.common import pixel_shuffle, normalize_01, normalize_m11, denormalize_m11
from model.srgan import generator, discriminator
from PIL import Image

Load the pre-trained weight from the model. You can download the pre-trained weight here
https://github.com/krasserm/super-resolution

In [3]:
pre_generator = generator()
gan_generator = generator()

pre_generator.load_weights('pre_trained_weight/weights/srgan/pre_generator.h5')
gan_generator.load_weights('pre_trained_weight/weights/srgan/gan_generator.h5')

In [4]:
def plot_sample(lr, sr):
    #plt.figure(dpi = 1500, frameon = False)
    #plt.imshow(sr)
    #plt.xticks([])
    #plt.yticks([])
    #plt.savefig('image_v1/test.png')
    
    # UNCOMMENT IF YOU WANT SIDE BY SIDE COMPARISON
    plt.figure(figsize=(20, 10), dpi = 1500)
    images = [lr, sr]
    titles = ['LR', 'SR']

    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, 2, i+1)
        plt.imshow(img)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])

In [5]:
def resolve_single(model, lr):
    return resolve(model, tf.expand_dims(lr, axis=0))[0]

In [6]:
def resolve(model, lr_batch):
    lr_batch = tf.cast(lr_batch, tf.float32)
    sr_batch = model(lr_batch)
    sr_batch = tf.clip_by_value(sr_batch, 0, 255)
    sr_batch = tf.round(sr_batch)
    sr_batch = tf.cast(sr_batch, tf.uint8)
    return sr_batch

In [11]:
# Comparison between the origin image and high resolution image
lr_image_path = 'image_v0/Floral_result_9.png'

model = generator()
model.load_weights('pre_trained_weight/weights/srgan/gan_generator.h5')

lr = Image.open(lr_image_path)
sr = resolve_single(model, lr)

plot_sample(lr, sr)

In [ ]:
# Section code to open all the files in a folder, generate high resolution images, and save all files to another folder
model = generator()
model.load_weights('pre_trained_weight/weights/srgan/gan_generator.h5')

path_location = 'images_before/ZigZag'

for filename in os.scandir(path_location):
    if filename.is_file():
        print(filename.name)
        lr = Image.open(filename.path)
        sr = resolve_single(model, lr)
        
        plt.figure(frameon = False)
        plt.imshow(sr)
        plt.xticks([])
        plt.yticks([])
        plt.savefig("images_after/ZigZag/" + filename.name)
        lr.close()